In [1]:
# !pip install scikit-image

In [10]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import namedtuple

### MY_Generator

In [12]:
class MY_Generator(tf.keras.utils.Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.image_filenames) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (200, 200))
               for file_name in batch_x]), np.array(batch_y)

In [ ]:
my_training_batch_generator = My_Generator(training_filenames, GT_training, batch_size)

### Second

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            # X[i,] = np.load('data/' + ID + '.npy')
            img_path = os.path.join('P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/' + ID + '.png')
            X[i,] = np.array(Image.open(img_path))
            
            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

### Explore label ids

In [14]:
from keras.preprocessing import image as image_utils

In [16]:
img = image_utils.img_to_array(image_utils.load_img('P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/aachen_000001_000019_gtFine_instanceIds.png'))

In [18]:
img2 = image_utils.img_to_array(image_utils.load_img('P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/aachen_000001_000019_gtFine_labelIds.png'))

In [20]:
# img
np.unique(img)

array([  0.,   1.,   3.,   4.,   5.,   6.,   7.,   8.,  11.,  12.,  13.,
        17.,  19.,  20.,  21.,  22.,  23., 255.], dtype=float32)

In [21]:
# img2
np.unique(img2)

array([ 0.,  1.,  3.,  4.,  5.,  6.,  7.,  8., 11., 12., 13., 17., 19.,
       20., 21., 22., 23., 26.], dtype=float32)

In [23]:
inventory = np.unique(img2, return_counts=True)

In [25]:
inventory[0]

array([ 0.,  1.,  3.,  4.,  5.,  6.,  7.,  8., 11., 12., 13., 17., 19.,
       20., 21., 22., 23., 26.], dtype=float32)

In [26]:
inventory[1]

array([    276,  244602,   94902,   62721,    1935,   87135, 2529687,
         76560, 1275681,   22422,   79071,   61956,     579,   10020,
        641403,  394923,  608991,   98592])

In [7]:
raw_data = pd.read_csv("raw_data.csv", sep='\t', encoding='utf-8', index_col=0)

In [8]:
raw_data.head()

,imgHeight,imgWidth,label,polygon,city,pic_id,set_type,target
0,1024,2048,sky,"[[126, 9], [235, 182], [797, 315], [1045, 339]...",lindau,000057_000019,val,sky
1,1024,2048,road,"[[37, 561], [362, 514], [811, 398], [955, 361]...",lindau,000057_000019,val,flat
2,1024,2048,building,"[[1140, 324], [1139, 334], [1138, 337], [1123,...",lindau,000057_000019,val,construction
3,1024,2048,vegetation,"[[1129, 376], [1127, 371], [1127, 369], [1125,...",lindau,000057_000019,val,nature
4,1024,2048,building,"[[1073, 337], [1072, 294], [1055, 291], [1049,...",lindau,000057_000019,val,construction


### Labels stuff

In [11]:
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

In [12]:
labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

### other stuff

In [16]:
df_labels = pd.DataFrame(labels, columns =['name', 'id', 'trainId', 'category', 'catId', 'hasInstances', 'ignoreInEval', 'color'])

In [26]:
df_labels.head(10)

,name,id,trainId,category,catId,hasInstances,ignoreInEval,color
0,unlabeled,0,255,void,0,False,True,"(0, 0, 0)"
1,ego vehicle,1,255,void,0,False,True,"(0, 0, 0)"
2,rectification border,2,255,void,0,False,True,"(0, 0, 0)"
3,out of roi,3,255,void,0,False,True,"(0, 0, 0)"
4,static,4,255,void,0,False,True,"(0, 0, 0)"
5,dynamic,5,255,void,0,False,True,"(111, 74, 0)"
6,ground,6,255,void,0,False,True,"(81, 0, 81)"
7,road,7,0,flat,1,False,False,"(128, 64, 128)"
8,sidewalk,8,1,flat,1,False,False,"(244, 35, 232)"
9,parking,9,255,flat,1,False,True,"(250, 170, 160)"


In [19]:
another_df = raw_data.merge(df_labels[['id', 'name']], how = 'left', left_on = 'label', right_on = 'name').drop(columns = ['name'])

In [21]:
another_df['id'] = another_df['id'].astype('Int64')

In [22]:
another_df

,imgHeight,imgWidth,label,polygon,city,pic_id,set_type,target,id,name
0,1024,2048,sky,"[[126, 9], [235, 182], [797, 315], [1045, 339]...",lindau,000057_000019,val,sky,23,sky
1,1024,2048,road,"[[37, 561], [362, 514], [811, 398], [955, 361]...",lindau,000057_000019,val,flat,7,road
2,1024,2048,building,"[[1140, 324], [1139, 334], [1138, 337], [1123,...",lindau,000057_000019,val,construction,11,building
3,1024,2048,vegetation,"[[1129, 376], [1127, 371], [1127, 369], [1125,...",lindau,000057_000019,val,nature,21,vegetation
4,1024,2048,building,"[[1073, 337], [1072, 294], [1055, 291], [1049,...",lindau,000057_000019,val,construction,11,building
...,...,...,...,...,...,...,...,...,...,...
287460,1024,2048,sidewalk,"[[2027, 803], [1895, 750], [1806, 712], [1774,...",weimar,000083_000019,train,flat,8,sidewalk
287461,1024,2048,pole,"[[1938, 709], [1929, 455], [1936, 448], [1928,...",weimar,000083_000019,train,object,17,pole
287462,1024,2048,terrain,"[[1913, 708], [1882, 658], [1878, 635], [1933,...",weimar,000083_000019,train,nature,22,terrain
287463,1024,2048,ego vehicle,"[[271, 1023], [387, 1009], [549, 993], [821, 9...",weimar,000083_000019,train,vehicle,1,ego vehicle


### Third

In [35]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
import tqdm
import cv2
from tensorflow.python.keras.callbacks import Callback

In [36]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

In [37]:
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}

In [47]:
mask_list

['aachen_000000_000019_gtFine_color.png',
 'aachen_000000_000019_gtFine_instanceIds.png',
 'aachen_000000_000019_gtFine_labelIds.png',
 'aachen_000000_000019_gtFine_polygons.json',
 'aachen_000001_000019_gtFine_color.png',
 'aachen_000001_000019_gtFine_instanceIds.png',
 'aachen_000001_000019_gtFine_labelIds.png',
 'aachen_000001_000019_gtFine_polygons.json',
 'aachen_000002_000019_gtFine_color.png',
 'aachen_000002_000019_gtFine_instanceIds.png',
 'aachen_000002_000019_gtFine_labelIds.png',
 'aachen_000002_000019_gtFine_polygons.json',
 'aachen_000003_000019_gtFine_color.png',
 'aachen_000003_000019_gtFine_instanceIds.png',
 'aachen_000003_000019_gtFine_labelIds.png',
 'aachen_000003_000019_gtFine_polygons.json',
 'aachen_000004_000019_gtFine_color.png',
 'aachen_000004_000019_gtFine_instanceIds.png',
 'aachen_000004_000019_gtFine_labelIds.png',
 'aachen_000004_000019_gtFine_polygons.json',
 'aachen_000005_000019_gtFine_color.png',
 'aachen_000005_000019_gtFine_instanceIds.png',
 'aac

In [49]:
image_dir = 'P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train/aachen'
mask_dir = 'P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen'
image_list = os.listdir(image_dir)
mask_list = os.listdir(mask_dir)
mask_list = [x for x in mask_list if x.endswith("labelIds.png")]
image_list.sort()
mask_list.sort()
print(f'. . . . .Number of images: {len(image_list)}\n. . . . .Number of masks: {len(mask_list)}')

# sanity check
# for i in range(len(image_list)):
#     assert image_list[i][16:] == mask_list[i][24:]


batch_size = 16
samples = 50000
steps = samples//batch_size
img_height, img_width = 256, 256
classes = 8
filters_n = 64

. . . . .Number of images: 174
. . . . .Number of masks: 174


In [41]:
class seg_gen(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.randint(0, 50000, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        for i in idx:
            _image = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}', target_size=(img_height, img_width)))/255.   
            img = image.img_to_array(image.load_img(f'{mask_dir}/{mask_list[i]}', grayscale=True, target_size=(img_height, img_width)))
            labels = np.unique(img)
            if len(labels) < 3:
                idx = np.random.randint(0, 50000, batch_size-drawn)
                continue
            img = np.squeeze(img)
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height*img_width, 8))
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1
        return np.array(batch_x), np.array(batch_y)

In [42]:
class visualize(Callback):
    def on_epoch_end(self, epoch, logs):
        print(f'\nGenerating output at epoch : {epoch}')
        i = 567
        img = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}'))/255.    
        dims = img.shape
        x = cv2.resize(img, (256, 256))
        x = np.float32(x)/255.
        z = unet.predict(np.expand_dims(x, axis=0))
        z = np.squeeze(z)
        z = z.reshape(256, 256, 8)
        z = cv2.resize(z, (dims[1], dims[0]))
        y = np.argmax(z, axis=2)
        
        construction = np.zeros_like(y)
        human = np.zeros_like(y)
        vehicle = np.zeros_like(y)
        construction[y==2] = 255.
        human[y==6] = 255.
        vehicle[y==7] = 255.
        
        result = img.copy()
        alpha = 0.4
        img[:,:,1] = construction
        img[:,:,2] = vehicle 
        img[:,:,0] = human

        cv2.addWeighted(img, alpha, result, 1-alpha, 0, result)
        cv2.imwrite(f'outputs/{epoch}.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        print('Wrote file to disk')

In [50]:
train_gen = seg_gen(image_list, mask_list, batch_size)

In [55]:
train_gen.__getitem__

<bound method seg_gen.__getitem__ of <__main__.seg_gen object at 0x7fcf61d9d370>>

In [ ]:
# unet = DilatedNet(256, 256, 8,use_ctx_module=True, bn=True)
# # p_unet = multi_gpu_model(unet, 4)
# # p_unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
# unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
# tb = TensorBoard(log_dir='logs', write_graph=True)
# mc = ModelCheckpoint(mode='max', filepath='models-dr/pdilated.h5', monitor='acc', save_best_only='True', save_weights_only='True', verbose=1)
# es = EarlyStopping(mode='max', monitor='acc', patience=6, verbose=1)
# vis = visualize()
# callbacks = [tb, mc, es]
# train_gen = seg_gen(image_list, mask_list, batch_size)

In [ ]:
# # p_unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
# unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
# print('Saving final weights')
# unet.save_weights('dilated.h5')